In [84]:
# !pip3 install openpyxl
import pandas as pd
import numpy as np
import re


### Data source is currently a CSV
### Later on this will be improved to be a webscraper or a data dump to an S3 bucket

In [85]:
df = pd.read_excel(r'UPR_cycle2.xlsx')
tags_matrix = pd.read_excel(r'themes_to_tags.xlsx')

In [86]:
df.index+=1
df.rename(columns={"Reccomending Body": "Recommending Body", "UPR Reccomending States": "UPR Recommending States"}, inplace=True)

# Data Cleaning

##### - Check unique number of text & annotation id to establish primary key
##### - Both have 625 unique entries which is the same as the total database entries
##### - Normalise by removing one of these, two identifiers is uneccesary we are not gaining anymore information

In [87]:
text_count = df.Text.unique().size
print(text_count)

625


In [88]:
annotation_id_count = df["OHCHR Annotation Id"].unique().size
print(annotation_id_count)

625


##### Columns to be dropped
##### Annotation ID, Recommending body, Text, Affected persons, Sdgs, Document publication date, session, date
##### check all entries are recommendation by seeing if text starts with number, then drop this column. 

In [89]:
drop_columns = ["OHCHR Annotation Id","Recommending Body", "Document Publication Date", "Sdgs", "UPR Session", "Date of publication on UHRI", "Affected Persons"]

In [90]:
df.drop(drop_columns, inplace=True, axis=1)

In [91]:
def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

In [92]:
df = df[df['Text'].apply(lambda x: has_numbers(x))]
df.drop(['Type'], inplace=True, axis=1)

In [93]:
df.index = df.index.set_names(['Recommendation_id'])

#### Standardize the naming convention, by removing hyphens and spaces, of all data entries.

In [94]:
def remove_leading_char(data_entry):
    """"Clean the naming convention for data entries of all columns"""
    clean_aff = re.sub(r'^[^A-Z]*', '', data_entry)
    if clean_aff:
        return clean_aff

In [95]:
def add_empty_cell_string(df):
    """Replacing all empty cells with default string 'Missing data'"""
    df.fillna('Missing data', inplace=True)
    return df 

In [96]:
df2 = add_empty_cell_string(df)

In [97]:
df2['Themes'] = df2['Themes'].str.split('\n')
df2 = df2.explode('Themes')

In [98]:
df3 = df2.applymap(lambda x: remove_leading_char(str(x)))

In [99]:
df4 = df3
df4['Recommending Regions'] = df4['Recommending Regions'].str.split('\n')
df4['UPR Recommending States'] = df4['UPR Recommending States'].str.split('\n')
df4 = df4.explode(['UPR Recommending States', 'Recommending Regions'])
df4 = df4.applymap(lambda x: remove_leading_char(str(x)))

In [100]:
Recommendations_df = df[['Text', 'UPR Position']].applymap(lambda x: remove_leading_char(str(x)))
Recommendations_df= Recommendations_df.reset_index(level=0)

In [101]:
Country_to_recommendations = df4[['Countries Concerned','UPR Recommending States']]
Country_to_recommendations.index = Country_to_recommendations.index.set_names(['Recommendation_id'])
Country_to_recommendations = Country_to_recommendations.reset_index(level=0).drop_duplicates(subset='Recommendation_id', keep='first')

In [102]:
c1 = df4[['Countries Concerned','Regions Concerned']].rename(columns={"Countries Concerned":"Country name", "Regions Concerned": "Regions"})
c2 = df4[['UPR Recommending States', 'Recommending Regions']].rename(columns={"UPR Recommending States":"Country name", "Recommending Regions": "Regions"})
Countries_df = pd.concat([c1,c2], ignore_index=True).drop_duplicates(subset=['Country name'], keep='first')
Countries_df = Countries_df.reset_index(drop='True')
Countries_df.index = Countries_df.index.set_names(['Country id'])
Countries_df.index+=1
Countries_df = Countries_df.reset_index()

In [103]:
Country_to_recommendations = Country_to_recommendations.merge(Countries_df[["Country name","Country id"]], left_on='Countries Concerned', right_on='Country name')
Country_to_recommendations = Country_to_recommendations.drop(columns=['Countries Concerned','Country name']).rename(columns={"Country id": "Countries Concerned"})
Country_to_recommendations = Country_to_recommendations.merge(Countries_df[["Country name","Country id"]], left_on='UPR Recommending States', right_on='Country name')
Country_to_recommendations = Country_to_recommendations.drop(columns=['UPR Recommending States','Country name']).rename(columns={"Country id": "UPR Recommending States"})

In [104]:
Regions_df = pd.DataFrame(Countries_df['Regions'].unique(), columns=['Regions'])
Regions_df.index = Regions_df.index.set_names(['Region id'])
Regions_df.index+=1
Regions_df = Regions_df.reset_index(level=0)

In [105]:
Countries_df = Countries_df.merge(Regions_df, left_on="Regions", right_on="Regions")
Countries_df = Countries_df.drop(columns=["Regions"])

In [106]:
Recommendation_to_theme = df4[['Themes']]
Recommendation_to_theme.index = Recommendation_to_theme.index.set_names(['Recommendation id'])
Recommendation_to_theme = Recommendation_to_theme.reset_index(level=0)

In [107]:
Themes_df = pd.DataFrame(df4['Themes'].unique(), columns=['Themes'])
Themes_df.index = Themes_df.index.set_names(['Theme id'])
Themes_df.index+=1
Themes_df = Themes_df.reset_index(level=0)


In [108]:
Recommendation_to_theme = Recommendation_to_theme.merge(Themes_df, left_on='Themes', right_on='Themes')
Recommendation_to_theme = Recommendation_to_theme.drop(columns=['Themes']).rename(columns={'Theme id':'Themes'})

### Normalized Dataframes

In [109]:
Recommendations_df

,Recommendation_id,Text,UPR Position
0,1,Promptly ratify the Optional Protocol to the C...,Noted
1,2,Ensure full and equal access to modern contrac...,Supported
2,3,Protect the rights of indigenous peoples throu...,Supported
3,4,Continue enhancing the school infrastructure f...,Supported
4,5,Continue implementing mechanisms for the prior...,Supported
...,...,...,...
620,621,Adopt legislation to guarantee the fulfilment ...,Supported
621,622,Establish clear consultation procedures in ord...,Noted
622,623,Consider ratifying ILO Convention N° 169 ( Nor...,Supported
623,624,Extend an invitation to the Working Group on e...,Supported


In [110]:
Country_to_recommendations

,Recommendation_id,Countries Concerned,UPR Recommending States
0,1,1,70
1,29,11,70
2,154,28,70
3,164,30,70
4,206,33,70
...,...,...,...
620,523,65,147
621,533,67,148
622,590,73,148
623,548,70,68


In [111]:
Countries_df

,Country id,Country name,Region id
0,1,Uganda,1
1,6,United Republic of Tanzania,1
2,7,Eswatini,1
3,15,Namibia,1
4,16,Sierra Leone,1
...,...,...,...
144,132,Saudi Arabia,5
145,133,Viet Nam,5
146,135,State of Palestine*,5
147,148,Oman,5


In [112]:
Regions_df

,Region id,Regions
0,1,Africa
1,2,Latin America and the Caribbean
2,3,Eastern Europe
3,4,Western Europe & Others
4,5,Asia-Pacific
5,6,Missing data


In [113]:
Recommendation_to_theme

,Recommendation id,Themes
0,1,1
1,9,1
2,11,1
3,16,1
4,35,1
...,...,...
1534,525,65
1535,543,66
1536,544,66
1537,589,67


In [114]:
Themes_df

,Theme id,Themes
0,1,Labour rights and right to work
1,2,Ratification of & accession to international i...
2,3,Prohibition of torture & ill-treatment (includ...
3,4,Sexual & reproductive health and rights
4,5,Right to participate in public affairs & right...
...,...,...
63,64,Good governance & corruption
64,65,Private life & privacy
65,66,Right to be recognized as a person before the law
66,67,Trade union rights


In [120]:
tags_df = pd.DataFrame(tags_matrix.columns,columns=['Tags'] )
tags_df.index = tags_df.index.set_names(['Tag id'])
tags_df

,Tags
Tag id,
0,Scope of international obligations and coopera...
1,National human rights framework
2,Development
3,Environment
4,Business and human rights
5,Human rights and counter-terrorism
6,Right to land
7,Civil & political rights - general measures of...
8,Equality and non-discrimination


In [121]:
tags_to_themes = pd.melt(tags_matrix, value_vars=tags_matrix.columns.tolist(), var_name='Tag', value_name='Theme')
tags_to_themes

,Tag,Theme
0,Scope of international obligations and coopera...,Scope of international obligations
1,Scope of international obligations and coopera...,Cooperation with human rights mechanisms & req...
2,Scope of international obligations and coopera...,Inter-State cooperation and assistance
3,Scope of international obligations and coopera...,Ratification of & accession to international i...
4,Scope of international obligations and coopera...,Reservations
...,...,...
391,Stateless persons,NaN
392,Stateless persons,NaN
393,Stateless persons,NaN
394,Stateless persons,NaN
